In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error
X_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
X_train.head()

In [ ]:
X_train[X_train.item_cnt_day<0]=0
X_train.item_cnt_day.unique()

In [ ]:
X_train.drop(['date_block_num','item_price'], axis=1, inplace=True)
X_train['date'] = pd.to_datetime(X_train['date'], dayfirst=True)
X_train['date'] = X_train['date'].apply(lambda x: x.strftime('%Y-%m'))
X_train.head()

In [ ]:
X = X_train.groupby(['date','shop_id','item_id']).sum()
X = X.pivot_table(index=['shop_id','item_id'], columns='date', values='item_cnt_day', fill_value=0)
X.reset_index(inplace=True)
X.head()

In [ ]:
Y = X['2015-10'].values
X_T = X.drop(['2015-10'], axis = 1)

x_train, x_test, y_train, y_test = train_test_split( X_T, Y, test_size=0.20, random_state=1)

In [ ]:
lin_reg = LinearRegression(normalize=True)
lin_reg.fit(x_train,y_train)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import numpy as np
def cross_val(model):
    pred = cross_val_score(model, X, Y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')

In [ ]:
test_pred = lin_reg.predict(x_test)
train_pred = lin_reg.predict(x_train)
print_evaluate(y_train, train_pred)
print_evaluate(y_test, test_pred)